Three types of backtesting of VaR:
- **Coverage tests** assess whether the frequency of exceedances is consistent with the quantile of loss a value-at-risk measure is intended to reflect.
- **Distribution tests** are goodness-of-fit tests applied to the overall loss distributions forecast by complete value-at-risk measures.
- **Independence tests** assess whether results appear to be independent from one period to the next.


**Dirty P&L’s and clean P&L’s**
Dirty P&L’s are the actual P&L’s reported for a portfolio by the accounting system. They can be impacted by trades that take place during the value-at-risk horizon—trades the value-at-risk measure cannot anticipate. Dirty P&L’s also reflect fee income earned during the value-at-risk horizon, which value-at-risk measures also don’t anticipate. Clean P&L’s are hypothetical P&L’s that would have been realized if no trading took place and no fee income were earned during the value-at-risk horizon.

https://www.youtube.com/watch?v=lc8q18FyZuU

In [163]:
# Coverage tests
import numpy as np

#1 Binomial test
import pandas as pd
data=pd.read_excel("Backtesting Data.xlsx",usecols=["VAR","Actual Loss","Hypothetical Loss"])
exceedances=data[data["Actual Loss"]>data["VAR"]]["VAR"]
exceedances_Actual=exceedances.count()
maxi=0.01*500+1.96*np.sqrt(500*0.01*.99)
mini=0.01*500-1.96*np.sqrt(500*0.01*.99)

#2 Kupiec Test - unconditional coverage test
pobs_Actual=(exceedances_Actual/500)
pexp=.05
n1=exceedances_Actual
n0=500-n1
LR=((pexp**n1)*((1-pexp)**n0))/((pobs_Actual**n1)*((1-pobs_Actual)**n0))
p_value=1-stats.chi2.cdf(x=-2*np.log(LR),df=1)
print(p_value,"- pvalue \n critical value",stats.chi2.ppf(0.95,1))

1.4125332669645552e-05 - pvalue 
 critical value 3.841458820694124


In [177]:
#Independence ( independence coverage tests ) 

#Christophersen Pelletier Test
exceedances_index=exceedances.index.values
n11=0
for i in range(len(exceedances_index)-1):
    if (exceedances_index[i+1]-exceedances_index[i])==1:
        n11+=1
n01=n1-n11
n10=n1-n11
n00=500-n01-n11
p00=n00/500
p01=n01/500
p10=n10/500
p11=n11/500

LRC=pow(pobs_Actual,n1)*pow(1-pobs_Actual,n0)/pow(p01,n01)/pow(1-p01,n00)/pow(p11,n11)/pow(1-p11,n10)
p_value=1-stats.chi2.cdf(x=-2*np.log(LRC),df=1)
print(p_value,"- pvalue \n critical value",stats.chi2.ppf(0.95,1))

#Conditional coverage test

LRC=pow(pexp,n1)*pow(1-pexp,n0)/pow(p01,n01)/pow(1-p01,n00)/pow(p11,n11)/pow(1-p11,n10)
p_value=1-stats.chi2.cdf(x=-2*np.log(LRC),df=2)
print(p_value,"- pvalue \n critical value",stats.chi2.ppf(0.95,1))

1.0 - pvalue 
 critical value 3.841458820694124
0.0007361389991606959 - pvalue 
 critical value 3.841458820694124


In [175]:
stats.chi2.cdf(-2*np.log(LRC),df=1)

0.0

In [176]:
import scipy.stats as stats
stats.chi2.ppf(q=0.1,df=1)


0.01579077409343122